In [ ]:
# 1. 安装必要的依赖
!pip install torch torchaudio yacs tqdm einops

# 2. 设置数据集路径
import os
os.makedirs('CASIA', exist_ok=True)
os.makedirs('ESD/ESV', exist_ok=True)

# 3. 复制数据集到工作目录
!cp -r /kaggle/input/casia/* CASIA/
!cp -r /kaggle/input/esd/* ESD/

# 4. 运行合并脚本
!python combine_metadata.py

!python extract_feature/WavLM/extract_wavlm.py

!python pretrain.py -M Vesper-12 -b 32 -g 0,1 -l 0.0005

# 5. 开始训练
!python finetune.py -M Vesper-12 -d combined -g 0 -b 32 -l 0.0007

# 监控GPU使用情况
!nvidia-smi

# 监控内存使用情况
!free -h

# 显示训练进度
from IPython.display import clear_output
import time

def monitor_training():
    while True:
        clear_output(wait=True)
        !nvidia-smi
        time.sleep(60)  # 每分钟更新一次

# 在后台运行监控
import threading
monitor_thread = threading.Thread(target=monitor_training)
monitor_thread.start()

# 保存模型和训练日志
!mkdir -p /kaggle/working/checkpoints
!cp -r checkpoints/* /kaggle/working/checkpoints/
!cp combined_metadata.csv /kaggle/working/